In [1]:
import string
import heapq
import collections
import random

In [2]:
words = [w.strip() for w in open('words7.txt').readlines()]
len(words)

9815

In [3]:
words[:10]

['abalone',
 'abandon',
 'abashed',
 'abashes',
 'abasing',
 'abating',
 'abdomen',
 'abducts',
 'abetted',
 'abetter']

In [4]:
def adjacents_explicit(word):
    neighbours = []
    for i in range(len(word)):
        for l in string.ascii_lowercase:
            if l != word[i]:
                neighbours.append(word[0:i] + l + word[i+1:])
    return neighbours

In [5]:
def adjacents(word):
    return [word[0:i] + l + word[i+1:]
           for i in range(len(word))
           for l in string.ascii_lowercase
           if l != word[i]]

In [10]:
# neighbours = {w: [n for n in adjacents(w) if n in words]
#              for w in words}

In [17]:
worddict = {w: True for w in words}
neighbours = {w: [n for n in adjacents(w) if n in worddict]
             for w in words}

In [12]:
# [w for w in words if sorted(neighbours[w]) != sorted(neighbours2[w])]

[]

In [7]:
# %%timeit
# neighbours = {w: [n for n in adjacents(w) if n in words]
#              for w in words}

1 loop, best of 3: 4min 2s per loop


In [8]:
%%timeit
worddict = {w: True for w in words}
neighbours2 = {w: [n for n in adjacents(w) if n in worddict]
             for w in words}

1 loop, best of 3: 655 ms per loop


In [13]:
%%timeit
wordset = set(words)
neighbours3 = {w: [n for n in adjacents(w) if n in wordset]
             for w in words}

1 loop, best of 3: 629 ms per loop


In [18]:
neighbours['abalone']

[]

In [20]:
neighbours['abashed']

['abashes']

In [21]:
neighbours['abasing']

['abusing', 'abating']

In [22]:
[(w, neighbours[w]) for w in words[:10]]

[('abalone', []),
 ('abandon', []),
 ('abashed', ['abashes']),
 ('abashes', ['abashed']),
 ('abasing', ['abusing', 'abating']),
 ('abating', ['abasing']),
 ('abdomen', []),
 ('abducts', []),
 ('abetted', ['abutted', 'abetter']),
 ('abetter', ['abettor', 'abetted'])]

In [37]:
[(w, neighbours[w]) for w in words[1000:1010]]

[('brewery', ['brewers']),
 ('brewing', ['crewing']),
 ('bribery', []),
 ('bribing', []),
 ('bricked', ['cricked', 'pricked', 'tricked', 'brisked']),
 ('bridals', []),
 ('bridged', ['bridled', 'bridges']),
 ('bridges', ['fridges', 'bridles', 'bridged']),
 ('bridled', ['bridged', 'bridles']),
 ('bridles', ['bridges', 'bridled'])]

In [23]:
def distance(w1, w2):
    return sum(1 for i in range(len(w1))
               if w1[i] != w2[i])

In [24]:
distance('abating', 'abating')

0

In [25]:
distance('abating', 'abetter')

4

In [26]:
def extend(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [38]:
extend(['bridges'])

[['bridges', 'fridges'], ['bridges', 'bridles'], ['bridges', 'bridged']]

In [39]:
extend(['bridges', 'bridles'])

[['bridges', 'bridles', 'bridled']]

In [40]:
extend(['bridges', 'bridles', 'bridled'])

[['bridges', 'bridles', 'bridled', 'bridged']]

In [69]:
def bfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [70]:
def bfs_search_closed(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [71]:
def dfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = successors + agenda[1:]
    if finished:
        return current
    else:
        return None        

In [72]:
def astar_search(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

In [73]:
def astar_search_closed(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    closed = set()
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

# Mutually-reachable sets

Find the transitive closure of the `neighbours` relation, so we can see which words can be transformed into which other words.

In [41]:
candidates = [set([k] + neighbours[k]) for k in neighbours]
reachables = []
while candidates:
    current = set(candidates.pop())
    altered = False
    for other in candidates:
        if current.intersection(other):
            altered = True
            current.update(other)
            candidates.remove(other)
    if altered:
        candidates.append(current)
    else:
        reachables.append(current)

len(reachables)

5109

In [42]:
len(max(reachables, key=len))

1400

In [43]:
len(min(reachables, key=len))

1

In [44]:
collections.Counter(len(r) for r in reachables)

Counter({1: 4102,
         2: 632,
         3: 183,
         4: 70,
         5: 35,
         6: 24,
         7: 19,
         8: 7,
         9: 11,
         10: 3,
         11: 5,
         12: 1,
         13: 4,
         14: 1,
         15: 1,
         17: 1,
         18: 1,
         19: 2,
         22: 1,
         25: 1,
         48: 1,
         133: 1,
         361: 1,
         773: 1,
         1400: 1})

In [45]:
sorted((r for r in reachables if len(r) > 1 if len(r) < 10), key=len)

[{'nullify', 'nullity'},
 {'believe', 'relieve'},
 {'wriggle', 'wriggly'},
 {'appeals', 'appears'},
 {'liaised', 'liaises'},
 {'gibbons', 'ribbons'},
 {'colonel', 'colones'},
 {'vehicle', 'vesicle'},
 {'unclean', 'unclear'},
 {'yodeled', 'yodeler'},
 {'minions', 'pinions'},
 {'achiest', 'ashiest'},
 {'regimen', 'regimes'},
 {'produce', 'product'},
 {'choicer', 'choices'},
 {'immured', 'immures'},
 {'retried', 'retries'},
 {'blessed', 'blesses'},
 {'herniae', 'hernias'},
 {'mealier', 'meatier'},
 {'scrubby', 'shrubby'},
 {'treacle', 'treadle'},
 {'redrawn', 'redraws'},
 {'modular', 'nodular'},
 {'lacunae', 'lacunas'},
 {'martial', 'partial'},
 {'jackals', 'jackass'},
 {'ploughs', 'sloughs'},
 {'salmons', 'saloons'},
 {'armored', 'armorer'},
 {'ability', 'agility'},
 {'draping', 'drawing'},
 {'tousled', 'tousles'},
 {'coerced', 'coerces'},
 {'fiestas', 'siestas'},
 {'rankled', 'rankles'},
 {'evolved', 'evolves'},
 {'maestri', 'maestro'},
 {'kennels', 'kernels'},
 {'donkeys', 'monkeys'},


In [46]:
'; '.join(sorted(list(r)[0] for r in reachables if len(r) == 1))

'abalone; abandon; abdomen; abducts; abiding; abolish; aborted; abounds; abreast; abridge; abscess; abscond; absence; absinth; absolve; absorbs; abstain; abusers; abusive; abysmal; abysses; acacias; academy; acanthi; acclaim; accords; accosts; account; accrual; accurst; acerbic; acetate; acetone; achieve; acolyte; aconite; acquire; acquits; acreage; acrider; acrobat; acronym; acrylic; actions; actives; actress; actuary; actuate; acutely; acutest; adagios; adamant; addicts; addling; address; adenoid; adeptly; adipose; adjoins; adjourn; adjudge; adjunct; adjusts; admiral; adoring; adorned; adrenal; adulate; advance; adverse; aerator; aerobic; aerosol; affairs; affirms; afflict; affords; affrays; affront; afghans; against; ageings; ageless; agendas; agilely; agilest; agitate; agonies; agonise; aground; aileron; ailment; airdrop; airfare; airhead; airings; airlift; airline; airmail; airport; airship; airsick; airways; alarmed; albinos; alchemy; alcohol; alcoves; alertly; alfalfa; algebra; 

In [47]:
for a in reachables:
    for b in reachables:
        if a != b:
            if not a.isdisjoint(b):
                print(a, b)

In [35]:
# longest_chain = []
# with open('all-chains-4.txt', 'w', 1) as f:
#     for ws in reachables:
#         for s in ws:
#             for t in ws:
#                 if s < t:
#                     chain = astar_search(s, t)
#                     if chain:
#                         f.write('{}\n'.format(chain))
#                         if len(chain) > len(longest_chain):
#                             longest_chain = chain

# longest_chain

In [48]:
bigset = max(reachables, key=len)

In [51]:
for _ in range(20):
    start, goal = random.sample(bigset, 2)
    print(astar_search_closed(start, goal))

['galling', 'gelling', 'selling', 'sealing', 'scaling', 'scaring']
['raining', 'railing', 'tailing', 'tabling', 'tabbing', 'gabbing', 'gobbing', 'bobbing', 'bobbins', 'bobbies', 'bobbles', 'babbles', 'gabbles', 'garbles', 'gargles', 'gaggles', 'giggles', 'wiggles']
['blowing', 'flowing', 'flawing', 'flaking', 'slaking', 'soaking', 'socking', 'sacking', 'tacking', 'tasking']
['bumbled', 'bubbled', 'bobbled', 'bobbles', 'bobbies', 'bobbins', 'bobbing', 'boobing', 'booting', 'boating', 'beating', 'beading', 'bending', 'pending']
['felling', 'belling', 'belting', 'bolting', 'booting', 'boobing', 'bobbing', 'bobbins', 'bobbies', 'bobbles', 'bubbles', 'burbles', 'burgles', 'bungles', 'bangles', 'tangles', 'tingles', 'tinkles', 'tickles']
['hurdled', 'huddled', 'huddles', 'puddles', 'paddles', 'paddies', 'daddies', 'dandies', 'dandier', 'handier', 'hardier', 'tardier', 'tarrier', 'terrier', 'tearier']
['seeding', 'sending', 'rending', 'renting', 'ranting', 'ratting', 'hatting']
['muddled', 'f

In [52]:
solutions = {}
for _ in range(10000):
    start, goal = random.sample(bigset, 2)
    solution = astar_search_closed(start, goal)
    sl = len(solution)
    if sl not in solutions:
        solutions[sl] = []
    if len(solutions[sl]) < 4:
        solutions[sl].append([start, goal])
        
#     if len(solution) >= 10:
#         solutions += [solution]
        
solutions

{2: [['soggier', 'doggier'],
  ['bashing', 'hashing'],
  ['growing', 'groping'],
  ['gulling', 'pulling']],
 3: [['middles', 'cuddles'],
  ['staring', 'snoring'],
  ['lashing', 'wishing'],
  ['reeking', 'peeping']],
 4: [['mulling', 'welling'],
  ['seeding', 'peering'],
  ['diddled', 'meddled'],
  ['wiggler', 'wangled']],
 5: [['yapping', 'bailing'],
  ['seating', 'pitying'],
  ['budging', 'gorging'],
  ['mailing', 'footing']],
 6: [['mooring', 'polling'],
  ['lapping', 'pocking'],
  ['rooking', 'slating'],
  ['palling', 'yucking']],
 7: [['polling', 'funding'],
  ['showing', 'jetting'],
  ['gonging', 'kenning'],
  ['tarring', 'tinting']],
 8: [['gabbing', 'fudging'],
  ['rubbing', 'forking'],
  ['zooming', 'railing'],
  ['humping', 'fouling']],
 9: [['soloing', 'gadding'],
  ['reefing', 'denying'],
  ['huffing', 'gearing'],
  ['gabbing', 'tensing']],
 10: [['touting', 'bumming'],
  ['loafing', 'kissing'],
  ['destiny', 'lording'],
  ['styling', 'dogging']],
 11: [['bagging', 'booties'

In [74]:
solutions = {}
for _ in range(10000):
    start, goal = random.sample(bigset, 2)
    solution = astar_search_closed(start, goal)
    if not solution:
        solution = astar_search_closed(goal, start)
    sl = len(solution)
    if sl > 30:
        if sl not in solutions:
            solutions[sl] = []
        if len(solutions[sl]) < 5:
            solutions[sl].append([start, goal])
        
#     if len(solution) >= 10:
#         solutions += [solution]
        
solutions

{31: [['pupping', 'lustier'],
  ['eloping', 'pastier'],
  ['daisies', 'poppies'],
  ['dossier', 'tattled'],
  ['betting', 'murkier']],
 32: [['getting', 'mossier'],
  ['busting', 'guppies'],
  ['mussier', 'staring'],
  ['coifing', 'murkier'],
  ['massing', 'cattier']],
 33: [['cattier', 'signing'],
  ['mousier', 'bulling'],
  ['tattles', 'railing'],
  ['rattler', 'tensing'],
  ['guppies', 'peeking']],
 34: [['tattled', 'sparing'],
  ['darting', 'bushier'],
  ['dunning', 'tattled'],
  ['rattles', 'deeding'],
  ['girting', 'luckier']],
 35: [['goading', 'battles'],
  ['rattles', 'griping'],
  ['jerkins', 'rattled'],
  ['wattled', 'pegging'],
  ['mintier', 'damming']],
 36: [['dotting', 'motives'],
  ['foiling', 'motives'],
  ['bottles', 'missing'],
  ['hussies', 'hulking'],
  ['letting', 'mottoes']],
 37: [['exuding', 'fattier'],
  ['tinting', 'mottoes'],
  ['bottled', 'temping'],
  ['mobiles', 'bending'],
  ['gushier', 'prising']],
 38: [['mobiles', 'walking'],
  ['motives', 'furring'],

In [75]:
solutions = {31: [['pupping', 'lustier'],
  ['eloping', 'pastier'],
  ['daisies', 'poppies'],
  ['dossier', 'tattled'],
  ['betting', 'murkier']],
 32: [['getting', 'mossier'],
  ['busting', 'guppies'],
  ['mussier', 'staring'],
  ['coifing', 'murkier'],
  ['massing', 'cattier']],
 33: [['cattier', 'signing'],
  ['mousier', 'bulling'],
  ['tattles', 'railing'],
  ['rattler', 'tensing'],
  ['guppies', 'peeking']],
 34: [['tattled', 'sparing'],
  ['darting', 'bushier'],
  ['dunning', 'tattled'],
  ['rattles', 'deeding'],
  ['girting', 'luckier']],
 35: [['goading', 'battles'],
  ['rattles', 'griping'],
  ['jerkins', 'rattled'],
  ['wattled', 'pegging'],
  ['mintier', 'damming']],
 36: [['dotting', 'motives'],
  ['foiling', 'motives'],
  ['bottles', 'missing'],
  ['hussies', 'hulking'],
  ['letting', 'mottoes']],
 37: [['exuding', 'fattier'],
  ['tinting', 'mottoes'],
  ['bottled', 'temping'],
  ['mobiles', 'bending'],
  ['gushier', 'prising']],
 38: [['mobiles', 'walking'],
  ['motives', 'furring'],
  ['bottled', 'yessing'],
  ['bottled', 'griming'],
  ['wormier', 'motives']],
 39: [['mottoes', 'reeving'],
  ['guiding', 'pussier'],
  ['pricing', 'cashier'],
  ['messier', 'arising'],
  ['playing', 'mobiles']],
 40: [['motives', 'jamming'],
  ['guppies', 'noising'],
  ['dimming', 'motiles'],
  ['chasing', 'mobiles'],
  ['poising', 'battled'],
  ['motives', 'smoking'],
  ['kissing', 'motives'],
  ['atoning', 'motives'],
  ['mossier', 'noising'],
  ['bottled', 'pricing']],
 41: [['priding', 'mottled']],
 42: [['eliding', 'mottoes'], ['poising', 'mottles']]}

In [55]:
len(astar_search_closed('mossier', 'noising'))

40

In [54]:
%%timeit
astar_search_closed('mossier', 'noising')

10 loops, best of 3: 154 ms per loop


In [77]:
len(astar_search_closed('eliding', 'mottoes'))

42

In [78]:
' '.join(astar_search_closed('eliding', 'mottoes'))

'eliding sliding slicing spicing spacing sparing searing bearing beating boating booting boobing bobbing bobbins bobbies bobbles bubbles burbles burgles bungles bangles dangles dandles dandies dandier handier hardier tardier tarrier tarries parries parties patties fatties fattier rattier rattler rattles battles bottles mottles mottoes'

In [76]:
%%timeit
astar_search_closed('eliding', 'mottoes')

10 loops, best of 3: 99.7 ms per loop


In [80]:
for l in sorted(solutions):
    print(', '.join(solutions[l][0]))

pupping, lustier
getting, mossier
cattier, signing
tattled, sparing
goading, battles
dotting, motives
exuding, fattier
mobiles, walking
mottoes, reeving
motives, jamming
priding, mottled
eliding, mottoes
